In [1]:
import sqlite3
#import pyodbc
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

The data set we are going to work with can be found in kaggle at 

https://www.kaggle.com/rtatman/188-million-us-wildfires

From kaggle we obtain the glossary bellow, that we also store into a dictionary:

Context:
This data publication contains a spatial database of wildfires that occurred in the United States from 1992 to 2015. It is the third update of a publication originally generated to support the national Fire Program Analysis (FPA) system. The wildfire records were acquired from the reporting systems of federal, state, and local fire organizations. The following core data elements were required for records to be included in this data publication: discovery date, final fire size, and a point location at least as precise as Public Land Survey System (PLSS) section (1-square mile grid). The data were transformed to conform, when possible, to the data standards of the National Wildfire Coordinating Group (NWCG). Basic error-checking was performed and redundant records were identified and removed, to the degree possible. The resulting product, referred to as the Fire Program Analysis fire-occurrence database (FPA FOD), includes 1.88 million geo-referenced wildfire records, representing a total of 140 million acres burned during the 24-year period.

Content:
This dataset is an SQLite database that contains the following information:

* Fires: Table including wildfire data for the period of 1992-2015 compiled from US federal, state, and local reporting systems.
* FOD_ID = Global unique identifier.
* FPA_ID = Unique identifier that contains information necessary to track back to the original record in the source dataset.
* SOURCESYSTEMTYPE = Type of source database or system that the record was drawn from (federal, nonfederal, or interagency).
* SOURCESYSTEM = Name of or other identifier for source database or system that the record was drawn from. See Table 1 in Short (2014), or \Supplements\FPAFODsourcelist.pdf, for a list of sources and their identifier.
* NWCGREPORTINGAGENCY = Active National Wildlife Coordinating Group (NWCG) Unit Identifier for the agency preparing the fire report (BIA = Bureau of Indian Affairs, BLM = Bureau of Land Management, BOR = Bureau of Reclamation, DOD = Department of Defense, DOE = Department of Energy, FS = Forest Service, FWS = Fish and Wildlife Service, IA = Interagency Organization, NPS = National Park Service, ST/C&L = State, County, or Local Organization, and TRIBE = Tribal Organization).
* NWCGREPORTINGUNIT_ID = Active NWCG Unit Identifier for the unit preparing the fire report.
* NWCGREPORTINGUNIT_NAME = Active NWCG Unit Name for the unit preparing the fire report.
* SOURCEREPORTINGUNIT = Code for the agency unit preparing the fire report, based on code/name in the source dataset.
* SOURCEREPORTINGUNIT_NAME = Name of reporting agency unit preparing the fire report, based on code/name in the source dataset.
* LOCALFIREREPORT_ID = Number or code that uniquely identifies an incident report for a particular reporting unit and a particular calendar year.
* LOCALINCIDENTID = Number or code that uniquely identifies an incident for a particular local fire management organization within a particular calendar year.
* FIRE_CODE = Code used within the interagency wildland fire community to track and compile cost information for emergency fire suppression (https://www.firecode.gov/).
* FIRE_NAME = Name of the incident, from the fire report (primary) or ICS-209 report (secondary).
* ICS209INCIDENT_NUMBER = Incident (event) identifier, from the ICS-209 report.
* ICS209NAME = Name of the incident, from the ICS-209 report.
* MTBS_ID = Incident identifier, from the MTBS perimeter dataset.
* MTBSFIRENAME = Name of the incident, from the MTBS perimeter dataset.
* COMPLEX_NAME = Name of the complex under which the fire was ultimately managed, when discernible.
* FIRE_YEAR = Calendar year in which the fire was discovered or confirmed to exist.
* DISCOVERY_DATE = Date on which the fire was discovered or confirmed to exist.
* DISCOVERY_DOY = Day of year on which the fire was discovered or confirmed to exist.
* DISCOVERY_TIME = Time of day that the fire was discovered or confirmed to exist.
* STATCAUSECODE = Code for the (statistical) cause of the fire.
* STATCAUSEDESCR = Description of the (statistical) cause of the fire.
* CONT_DATE = Date on which the fire was declared contained or otherwise controlled (mm/dd/yyyy where mm=month, dd=day, and yyyy=year).
* CONT_DOY = Day of year on which the fire was declared contained or otherwise controlled.
* CONT_TIME = Time of day that the fire was declared contained or otherwise controlled (hhmm where hh=hour, mm=minutes).
* FIRE_SIZE = Estimate of acres within the final perimeter of the fire.
* FIRESIZECLASS = Code for fire size based on the number of acres within the final fire perimeter expenditures (A=greater than 0 but less than or equal to 0.25 acres, B=0.26-9.9 acres, C=10.0-99.9 acres, D=100-299 acres, E=300 to 999 acres, F=1000 to 4999 acres, and G=5000+ acres).
* LATITUDE = Latitude (NAD83) for point location of the fire (decimal degrees).
* LONGITUDE = Longitude (NAD83) for point location of the fire (decimal degrees).
* OWNER_CODE = Code for primary owner or entity responsible for managing the land at the point of origin of the fire at the time of the incident.
* OWNER_DESCR = Name of primary owner or entity responsible for managing the land at the point of origin of the fire at the time of the incident.
* STATE = Two-letter alphabetic code for the state in which the fire burned (or originated), based on the nominal designation in the fire report.
* COUNTY = County, or equivalent, in which the fire burned (or originated), based on nominal designation in the fire report.
* FIPS_CODE = Three-digit code from the Federal Information Process Standards (FIPS) publication 6-4 for representation of counties and equivalent entities.
* FIPS_NAME = County name from the FIPS publication 6-4 for representation of counties and equivalent entities.
* NWCGUnitIDActive20170109: Look-up table containing all NWCG identifiers for agency units that were active (i.e., valid) as of 9 January 2017, when the list was downloaded from https://www.nifc.blm.gov/unit_id/Publish.html and used as the source of values available to populate the following fields in the Fires table: NWCGREPORTINGAGENCY, NWCGREPORTINGUNITID, and NWCGREPORTINGUNITNAME.
* UnitId = NWCG Unit ID.
* GeographicArea = Two-letter code for the geographic area in which the unit is located (NA=National, IN=International, AK=Alaska, CA=California, EA=Eastern Area, GB=Great Basin, NR=Northern Rockies, NW=Northwest, RM=Rocky Mountain, SA=Southern Area, and SW=Southwest).
* Gacc = Seven or eight-letter code for the Geographic Area Coordination Center in which the unit is located or primarily affiliated with (CAMBCIFC=Canadian Interagency Forest Fire Centre, USAKCC=Alaska Interagency Coordination Center, USCAONCC=Northern California Area Coordination Center, USCAOSCC=Southern California Coordination Center, USCORMCC=Rocky Mountain Area Coordination Center, USGASAC=Southern Area Coordination Center, USIDNIC=National Interagency Coordination Center, USMTNRC=Northern Rockies Coordination Center, USNMSWC=Southwest Area Coordination Center, USORNWC=Northwest Area Coordination Center, USUTGBC=Western Great Basin Coordination Center, USWIEACC=Eastern Area Coordination Center).
* WildlandRole = Role of the unit within the wildland fire community.
* UnitType = Type of unit (e.g., federal, state, local).
* Department = Department (or state/territory) to which the unit belongs (AK=Alaska, AL=Alabama, AR=Arkansas, AZ=Arizona, CA=California, CO=Colorado, CT=Connecticut, DE=Delaware, DHS=Department of Homeland Security, DOC= Department of Commerce, DOD=Department of Defense, DOE=Department of Energy, DOI= Department of Interior, DOL=Department of Labor, FL=Florida, GA=Georgia, IA=Iowa, IA/GC=Non-Departmental Agencies, ID=Idaho, IL=Illinois, IN=Indiana, KS=Kansas, KY=Kentucky, LA=Louisiana, MA=Massachusetts, MD=Maryland, ME=Maine, MI=Michigan, MN=Minnesota, MO=Missouri, MS=Mississippi, MT=Montana, NC=North Carolina, NE=Nebraska, NG=Non-Government, NH=New Hampshire, NJ=New Jersey, NM=New Mexico, NV=Nevada, NY=New York, OH=Ohio, OK=Oklahoma, OR=Oregon, PA=Pennsylvania, PR=Puerto Rico, RI=Rhode Island, SC=South Carolina, SD=South Dakota, ST/L=State or Local Government, TN=Tennessee, Tribe=Tribe, TX=Texas, USDA=Department of Agriculture, UT=Utah, VA=Virginia, VI=U. S. Virgin Islands, VT=Vermont, WA=Washington, WI=Wisconsin, WV=West Virginia, WY=Wyoming).
* Agency = Agency or bureau to which the unit belongs (AG=Air Guard, ANC=Alaska Native Corporation, BIA=Bureau of Indian Affairs, BLM=Bureau of Land Management, BOEM=Bureau of Ocean Energy Management, BOR=Bureau of Reclamation, BSEE=Bureau of Safety and Environmental Enforcement, C&L=County & Local, CDF=California Department of Forestry & Fire Protection, DC=Department of Corrections, DFE=Division of Forest Environment, DFF=Division of Forestry Fire & State Lands, DFL=Division of Forests and Land, DFR=Division of Forest Resources, DL=Department of Lands, DNR=Department of Natural Resources, DNRC=Department of Natural Resources and Conservation, DNRF=Department of Natural Resources Forest Service, DOA=Department of Agriculture, DOC=Department of Conservation, DOE=Department of Energy, DOF=Department of Forestry, DVF=Division of Forestry, DWF=Division of Wildland Fire, EPA=Environmental Protection Agency, FC=Forestry Commission, FEMA=Federal Emergency Management Agency, FFC=Bureau of Forest Fire Control, FFP=Forest Fire Protection, FFS=Forest Fire Service, FR=Forest Rangers, FS=Forest Service, FWS=Fish & Wildlife Service, HQ=Headquarters, JC=Job Corps, NBC=National Business Center, NG=National Guard, NNSA=National Nuclear Security Administration, NPS=National Park Service, NWS=National Weather Service, OES=Office of Emergency Services, PRI=Private, SF=State Forestry, SFS=State Forest Service, SP=State Parks, TNC=The Nature Conservancy, USA=United States Army, USACE=United States Army Corps of Engineers, USAF=United States Air Force, USGS=United States Geological Survey, USN=United States Navy).
* Parent = Agency subgroup to which the unit belongs (A concatenation of State and Unit from this report - https://www.nifc.blm.gov/unit_id/publish/UnitIdReport.rtf).
* Country = Country in which the unit is located (e.g. US = United States).
* State = Two-letter code for the state in which the unit is located (or primarily affiliated).
* Code = Unit code (follows state code to create UnitId).
* Name = Unit name.

Questions we would like to answer based on the original kaggle questions: 
* Have wildfires become more or less frequent over time?
* Given the size, location and date, and some other information, can you predict the cause of a fire wildfire?

In [2]:
# Below we create a dictionary that contains the abbreviations given above. The intention of this dictionary is to have those abbreviations at hand.

abb = {}

abb['Fires'] = 'Table including wildfire data for the period of 1992-2015 compiled from US federal, state, and local reporting systems'



abb['FOD_ID'] = 'Global unique identifier'


abb['FPA_ID'] = 'Unique identifier that contains information necessary to track back to the original record in the source dataset'

abb['SOURCE_SYSTEM_TYPE'] = 'Type of source database or system that the record was drawn from (federal, nonfederal, or interagency)'

abb['SOURCE_SYSTEM'] = 'Name of or other identifier for source database or system that the record was drawn from. See Table 1 in Short (2014), or \Supplements\FPAFODsourcelist.pdf, for a list of sources and their identifier'

abb['NWCG_REPORTING_AGENCY'] = 'Active National Wildlife Coordinating Group (NWCG) Unit Identifier for the agency preparing the fire report (BIA = Bureau of Indian Affairs, BLM = Bureau of Land Management, BOR = Bureau of Reclamation, DOD = Department of Defense, DOE = Department of Energy, FS = Forest Service, FWS = Fish and Wildlife Service, IA = Interagency Organization, NPS = National Park Service, ST/C&L = State, County, or Local Organization, and TRIBE = Tribal Organization)'

abb['NWCG_REPORTING_UNIT_ID'] = 'Active NWCG Unit Identifier for the unit preparing the fire report'

abb['NWCG_REPORTING_UNIT_NAME'] = 'Active NWCG Unit Name for the unit preparing the fire report'

abb['SOURCE_REPORTING_UNIT'] = 'Code for the agency unit preparing the fire report, based on code/name in the source dataset'

abb['SOURCE_REPORTING_UNIT_NAME'] = 'Name of reporting agency unit preparing the fire report, based on code/name in the source dataset'

abb['LOCALFIREREPORT_ID'] = 'Number or code that uniquely identifies an incident report for a particular reporting unit and a particular calendar year'

abb['LOCALINCIDENTID'] = 'Number or code that uniquely identifies an incident for a particular local fire management organization within a particular calendar year'

abb['FIRE_CODE'] ='Code used within the interagency wildland fire community to track and compile cost information for emergency fire suppression (https://www.firecode.gov/)'

abb['FIRE_NAME'] = 'Name of the incident, from the fire report (primary) or ICS-209 report (secondary)'

abb['ICS209INCIDENT_NUMBER'] = 'Incident (event) identifier, from the ICS-209 report'

abb['ICS209NAME'] = 'Name of the incident, from the ICS-209 report'

abb['MTBS_ID'] = 'Incident identifier, from the MTBS perimeter dataset'

abb['MTBSFIRENAME'] = 'Name of the incident, from the MTBS perimeter dataset'

abb['COMPLEX_NAME'] = 'Name of the complex under which the fire was ultimately managed, when discernible'

abb['FIRE_YEAR'] = 'Calendar year in which the fire was discovered or confirmed to exist'

abb['DISCOVERY_DATE'] ='Date on which the fire was discovered or confirmed to exist'

abb['DISCOVERY_DOY'] = 'Day of year on which the fire was discovered or confirmed to exist'

abb['DISCOVERY_TIME'] = 'Time of day that the fire was discovered or confirmed to exist'

abb['STAT_CAUSE_CODE'] = 'Code for the (statistical) cause of the fire'

abb['STAT_CAUSE_DESCR'] = 'Description of the (statistical) cause of the fire'

abb['CONT_DATE'] = 'Date on which the fire was declared contained or otherwise controlled (mm/dd/yyyy where mm=month, dd=day, and yyyy=year)'

abb['CONT_DOY'] = 'Day of year on which the fire was declared contained or otherwise controlled'

abb['CONT_TIME'] = 'Time of day that the fire was declared contained or otherwise controlled (hhmm where hh=hour, mm=minutes)'

abb['FIRE_SIZE'] = 'Estimate of acres within the final perimeter of the fire'

abb['FIRE_SIZE_CLASS'] = 'Code for fire size based on the number of acres within the final fire perimeter expenditures (A=greater than 0 but less than or equal to 0.25 acres, B=0.26-9.9 acres, C=10.0-99.9 acres, D=100-299 acres, E=300 to 999 acres, F=1000 to 4999 acres, and G=5000+ acres)'

abb['LATITUDE'] = 'Latitude (NAD83) for point location of the fire (decimal degrees)'

abb['LONGITUDE'] = 'Longitude (NAD83) for point location of the fire (decimal degrees)'

abb['OWNER_CODE'] = 'Code for primary owner or entity responsible for managing the land at the point of origin of the fire at the time of the incident'

abb['OWNER_DESCR'] = 'Name of primary owner or entity responsible for managing the land at the point of origin of the fire at the time of the incident'

abb['STATE'] = 'Two-letter alphabetic code for the state in which the fire burned (or originated), based on the nominal designation in the fire report'

abb['COUNTY'] = 'County, or equivalent, in which the fire burned (or originated), based on nominal designation in the fire report'

abb['FIPS_CODE'] = 'Three-digit code from the Federal Information Process Standards (FIPS) publication 6-4 for representation of counties and equivalent entities'

abb['FIPS_NAME'] = 'County name from the FIPS publication 6-4 for representation of counties and equivalent entities'

abb['NWCGUnitIDActive20170109'] = 'Look-up table containing all NWCG identifiers for agency units that were active (i.e., valid) as of 9 January 2017, when the list was downloaded from https://www.nifc.blm.gov/unit_id/Publish.html and used as the source of values available to populate the following fields in the Fires table: NWCGREPORTINGAGENCY, NWCGREPORTINGUNITID, and NWCGREPORTINGUNITNAME'

abb['UnitId'] = 'NWCG Unit ID'

abb['GeographicArea'] = 'Two-letter code for the geographic area in which the unit is located (NA=National, IN=International, AK=Alaska, CA=California, EA=Eastern Area, GB=Great Basin, NR=Northern Rockies, NW=Northwest, RM=Rocky Mountain, SA=Southern Area, and SW=Southwest)'

abb['Gacc']= 'Seven or eight-letter code for the Geographic Area Coordination Center in which the unit is located or primarily affiliated with (CAMBCIFC=Canadian Interagency Forest Fire Centre, USAKCC=Alaska Interagency Coordination Center, USCAONCC=Northern California Area Coordination Center, USCAOSCC=Southern California Coordination Center, USCORMCC=Rocky Mountain Area Coordination Center, USGASAC=Southern Area Coordination Center, USIDNIC=National Interagency Coordination Center, USMTNRC=Northern Rockies Coordination Center, USNMSWC=Southwest Area Coordination Center, USORNWC=Northwest Area Coordination Center, USUTGBC=Western Great Basin Coordination Center, USWIEACC=Eastern Area Coordination Center)'

abb['WildlandRole'] = 'Role of the unit within the wildland fire community'

abb['UnitType'] = 'Type of unit (e.g., federal, state, local)'

abb['Department'] = 'Department (or state/territory) to which the unit belongs (AK=Alaska, AL=Alabama, AR=Arkansas, AZ=Arizona, CA=California, CO=Colorado, CT=Connecticut, DE=Delaware, DHS=Department of Homeland Security, DOC= Department of Commerce, DOD=Department of Defense, DOE=Department of Energy, DOI= Department of Interior, DOL=Department of Labor, FL=Florida, GA=Georgia, IA=Iowa, IA/GC=Non-Departmental Agencies, ID=Idaho, IL=Illinois, IN=Indiana, KS=Kansas, KY=Kentucky, LA=Louisiana, MA=Massachusetts, MD=Maryland, ME=Maine, MI=Michigan, MN=Minnesota, MO=Missouri, MS=Mississippi, MT=Montana, NC=North Carolina, NE=Nebraska, NG=Non-Government, NH=New Hampshire, NJ=New Jersey, NM=New Mexico, NV=Nevada, NY=New York, OH=Ohio, OK=Oklahoma, OR=Oregon, PA=Pennsylvania, PR=Puerto Rico, RI=Rhode Island, SC=South Carolina, SD=South Dakota, ST/L=State or Local Government, TN=Tennessee, Tribe=Tribe, TX=Texas, USDA=Department of Agriculture, UT=Utah, VA=Virginia, VI=U. S. Virgin Islands, VT=Vermont, WA=Washington, WI=Wisconsin, WV=West Virginia, WY=Wyoming)'

abb['Agency'] = 'Agency or bureau to which the unit belongs (AG=Air Guard, ANC=Alaska Native Corporation, BIA=Bureau of Indian Affairs, BLM=Bureau of Land Management, BOEM=Bureau of Ocean Energy Management, BOR=Bureau of Reclamation, BSEE=Bureau of Safety and Environmental Enforcement, C&L=County & Local, CDF=California Department of Forestry & Fire Protection, DC=Department of Corrections, DFE=Division of Forest Environment, DFF=Division of Forestry Fire & State Lands, DFL=Division of Forests and Land, DFR=Division of Forest Resources, DL=Department of Lands, DNR=Department of Natural Resources, DNRC=Department of Natural Resources and Conservation, DNRF=Department of Natural Resources Forest Service, DOA=Department of Agriculture, DOC=Department of Conservation, DOE=Department of Energy, DOF=Department of Forestry, DVF=Division of Forestry, DWF=Division of Wildland Fire, EPA=Environmental Protection Agency, FC=Forestry Commission, FEMA=Federal Emergency Management Agency, FFC=Bureau of Forest Fire Control, FFP=Forest Fire Protection, FFS=Forest Fire Service, FR=Forest Rangers, FS=Forest Service, FWS=Fish & Wildlife Service, HQ=Headquarters, JC=Job Corps, NBC=National Business Center, NG=National Guard, NNSA=National Nuclear Security Administration, NPS=National Park Service, NWS=National Weather Service, OES=Office of Emergency Services, PRI=Private, SF=State Forestry, SFS=State Forest Service, SP=State Parks, TNC=The Nature Conservancy, USA=United States Army, USACE=United States Army Corps of Engineers, USAF=United States Air Force, USGS=United States Geological Survey, USN=United States Navy)'

abb['Parent'] = 'Agency subgroup to which the unit belongs (A concatenation of State and Unit from this report - https://www.nifc.blm.gov/unit_id/publish/UnitIdReport.rtf)'

abb['Country'] = 'Country in which the unit is located (e.g. US = United States)'

abb['State'] = 'Two-letter code for the state in which the unit is located (or primarily affiliated)'

abb['Code'] = 'Unit code (follows state code to create UnitId)'

abb['Name'] = 'Unit name'

Below we are making a connection to get the data

In [3]:
# Let's make the connection to get the data
conn = sqlite3.connect('C:\\Users\\francisco\\Desktop\\BrainStation\\Capstoneproject_BS\\FiresUSA\\FPA_FOD_20170508.sqlite')

Now we are going to see the information contained in our database.

In [4]:
# we can get all the info using the below command:
pd.read_sql('SELECT * FROM sqlite_master;', conn)

,type,name,tbl_name,rootpage,sql
0,table,spatial_ref_sys,spatial_ref_sys,2,CREATE TABLE spatial_ref_sys (\nsrid INTEGER N...
1,index,idx_spatial_ref_sys,spatial_ref_sys,3,CREATE UNIQUE INDEX idx_spatial_ref_sys \nON s...
2,table,spatialite_history,spatialite_history,4,CREATE TABLE spatialite_history (\nevent_id IN...
3,table,sqlite_sequence,sqlite_sequence,5,"CREATE TABLE sqlite_sequence(name,seq)"
4,table,geometry_columns,geometry_columns,6,CREATE TABLE geometry_columns (\nf_table_name ...
...,...,...,...,...,...
116,table,idx_Fires_Shape,idx_Fires_Shape,0,"CREATE VIRTUAL TABLE ""idx_Fires_Shape"" USING r..."
117,table,idx_Fires_Shape_node,idx_Fires_Shape_node,677301,"CREATE TABLE ""idx_Fires_Shape_node""(nodeno INT..."
118,table,idx_Fires_Shape_rowid,idx_Fires_Shape_rowid,677302,"CREATE TABLE ""idx_Fires_Shape_rowid""(rowid INT..."
119,table,idx_Fires_Shape_parent,idx_Fires_Shape_parent,677303,"CREATE TABLE ""idx_Fires_Shape_parent""(nodeno I..."


Let's focus on the tables

In [5]:
# Let's explore some tables

tbl_names = pd.read_sql('SELECT * FROM sqlite_master WHERE type="table";', conn)['name']
tbl_names

#queries = []
#for n in tbl_names:
#    queries.append(f"SELECT * FROM {n};")
#for n in range(len(queries)):
#    pd.read_sql(queries[n], conn).head()

0                        spatial_ref_sys
1                     spatialite_history
2                        sqlite_sequence
3                       geometry_columns
4                    spatial_ref_sys_aux
5                 views_geometry_columns
6                 virts_geometry_columns
7            geometry_columns_statistics
8      views_geometry_columns_statistics
9      virts_geometry_columns_statistics
10          geometry_columns_field_infos
11    views_geometry_columns_field_infos
12    virts_geometry_columns_field_infos
13                 geometry_columns_time
14                 geometry_columns_auth
15           views_geometry_columns_auth
16           virts_geometry_columns_auth
17                    sql_statements_log
18                          SpatialIndex
19                  ElementaryGeometries
20                                   KNN
21                                 Fires
22                       idx_Fires_Shape
23                  idx_Fires_Shape_node
24              

Table 21: ``Fires`` is the relevant table for us, as indicated by the original source. However, we will take a look at the tables contained in this dataset.

To that end, we can use a for loop using the query below

``myquery = f'SELECT COUNT(*) FROM {tbl_names[i]};'``

where i runs from 0 to 26. After that, we can read the table's content with 

``pd.read_sql(myquery, conn)``

We will not use the content of some tables as they are indices (like numer 18)

In [6]:
tbl_ind = list(range(27))

index_tables = [18, 19]

index_tables = [18, 19, 20, 22]

for i in index_tables:
    tbl_ind.remove(i)

#tbl_ind

In [7]:
%%time
#Since we have a considerable large amount of data, we will only read the first few rowa of our tables.


for i in tbl_ind:
    myquery = f'SELECT * FROM {tbl_names[i]} LIMIT 6;'

    table = pd.read_sql(myquery, conn)
    
    print(f'Content of table {tbl_names[i]}.')
    print(f'The index is {i}')
    display(table)
    # To visualize the data in a better way we are leaveing some space between the tables
    print(' ')
    print(' ')
    print(' ')
    print(' ')
    print(' ')

Content of table spatial_ref_sys.
The index is 0


,srid,auth_name,auth_srid,ref_sys_name,proj4text,srtext
0,-1,NONE,-1,Undefined - Cartesian,,Undefined
1,0,NONE,0,Undefined - Geographic Long/Lat,,Undefined
2,2000,epsg,2000,Anguilla 1957 / British West Indies Grid,+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.999500000...,"PROJCS[""Anguilla 1957 / British West Indies Gr..."
3,2001,epsg,2001,Antigua 1943 / British West Indies Grid,+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.999500000...,"PROJCS[""Antigua 1943 / British West Indies Gri..."
4,2002,epsg,2002,Dominica 1945 / British West Indies Grid,+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.999500000...,"PROJCS[""Dominica 1945 / British West Indies Gr..."
5,2003,epsg,2003,Grenada 1953 / British West Indies Grid,+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.999500000...,"PROJCS[""Grenada 1953 / British West Indies Gri..."


 
 
 
 
 
Content of table spatialite_history.
The index is 1


,event_id,table_name,geometry_column,event,timestamp,ver_sqlite,ver_splite
0,1,spatial_ref_sys,None,table successfully created,2017-05-17T22:00:26.772Z,3.9.2,4.4.0-RC0
1,2,geometry_columns,None,table successfully created,2017-05-17T22:00:26.796Z,3.9.2,4.4.0-RC0
2,3,geometry_columns,None,trigger 'geometry_columns_f_table_name_insert'...,2017-05-17T22:00:26.796Z,3.9.2,4.4.0-RC0
3,4,geometry_columns,None,trigger 'geometry_columns_f_table_name_update'...,2017-05-17T22:00:26.796Z,3.9.2,4.4.0-RC0
4,5,geometry_columns,None,trigger 'geometry_columns_f_geometry_column_in...,2017-05-17T22:00:26.797Z,3.9.2,4.4.0-RC0
5,6,geometry_columns,None,trigger 'geometry_columns_f_geometry_column_up...,2017-05-17T22:00:26.797Z,3.9.2,4.4.0-RC0


 
 
 
 
 
Content of table sqlite_sequence.
The index is 2


...

 
 
 
 
 
Content of table idx_Fires_Shape_parent.
The index is 25


,nodeno,parentnode
0,2,44163
1,3,2235
2,4,56494
3,5,63763
4,6,81
5,7,63241


 
 
 
 
 
Content of table NWCG_UnitIDActive_20170109.
The index is 26


,OBJECTID,UnitId,GeographicArea,Gacc,WildlandRole,UnitType,Department,Agency,Parent,Country,State,Code,Name
0,1,USAKCAN,AK,USAKACC,Resource Provider Only,Non-Government,NG,ANC,None,US,AK,CAN,Calista Regional Native Corporation
1,2,USAKCEKX,AK,USAKACC,Incident Host Geographic,US County/Local,AK,C&L,None,US,AK,CEKX,Central Emergency Services
2,3,USAKCFFX,AK,USAKACC,Incident Host Geographic,US County/Local,AK,C&L,None,US,AK,CFFX,City Fairbanks Fire Department
3,4,USAKCFMX,AK,USAKACC,Incident Host Geographic,US County/Local,AK,C&L,None,US,AK,CFMX,Central Mat-Su Fire Department
4,5,USAKCFQ,AK,USAKACC,Incident Host Geographic,US Federal,DOD,USAF,None,US,AK,CFQ,Clear Air Force Station
5,6,USAKCGF,AK,USAKACC,Incident Host Geographic,US Federal,USDA,FS,None,US,AK,CGF,Chugach National Forest


 
 
 
 
 
Wall time: 671 ms


We will focus now on table 21: ``Fires``. 

In [43]:
%%time
# Let's see how many datapoints we have

myquery_count = f'SELECT COUNT(*) FROM {tbl_names[21]};'

count = pd.read_sql(myquery_count, conn)
print(count)

   COUNT(*)
0   1880465
Wall time: 7.38 s


We have about $1.8$M observations. We will read in the data below. Note that we are limiting to exactly $1.8$M observations. The reason for that is that the machine I am working on, collapses when I try to read in all the data. Therefore, we are going to work with  $96\%$ of the original data (see the next cell where the percentage is calculted).

In [45]:
1800000/1880465

0.9572100517691103

In [8]:
%%time
#The code commented below was used to read in a subset of the data set to get some initial insights


limit = 1800000

myquery = f'SELECT * FROM {tbl_names[21]} LIMIT {limit};'

fires_df = pd.read_sql(myquery, conn)


#myquery = f'SELECT * FROM {tbl_names[21]};'

#fires_df = pd.read_sql(myquery, conn)

Wall time: 5min 18s


Let's see what is contained in the data

In [9]:
# What's contained in the data?

fires_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800000 entries, 0 to 1799999
Data columns (total 39 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   OBJECTID                    int64  
 1   FOD_ID                      int64  
 2   FPA_ID                      object 
 3   SOURCE_SYSTEM_TYPE          object 
 4   SOURCE_SYSTEM               object 
 5   NWCG_REPORTING_AGENCY       object 
 6   NWCG_REPORTING_UNIT_ID      object 
 7   NWCG_REPORTING_UNIT_NAME    object 
 8   SOURCE_REPORTING_UNIT       object 
 9   SOURCE_REPORTING_UNIT_NAME  object 
 10  LOCAL_FIRE_REPORT_ID        object 
 11  LOCAL_INCIDENT_ID           object 
 12  FIRE_CODE                   object 
 13  FIRE_NAME                   object 
 14  ICS_209_INCIDENT_NUMBER     object 
 15  ICS_209_NAME                object 
 16  MTBS_ID                     object 
 17  MTBS_FIRE_NAME              object 
 18  COMPLEX_NAME                object 
 19  FIRE_YEAR            

In [10]:
%%time
# We can also take a look at the description

fires_df.describe(include='all')

Wall time: 26.9 s
Parser   : 479 ms


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
count,1.800000e+06,1.800000e+06,1800000,1800000,1800000,1800000,1800000,1800000,1800000,1800000,...,1800000,1.800000e+06,1.800000e+06,1.800000e+06,1800000,1800000,1137293,1137293,1137293,1800000
unique,NaN,NaN,1799998,3,37,11,1620,1615,4825,4380,...,7,NaN,NaN,NaN,16,52,3308,284,1665,1498762
top,NaN,NaN,ICS209_2009_KS-DDQ-128,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,B,NaN,NaN,NaN,MISSING/NOT SPECIFIED,CA,5,005,Washington,b'\x00\x01\xad\x10\x00\x00\xb0\xd19?\xc5\x8fP\...
freq,NaN,NaN,2,1296771,654019,1301619,164811,164811,95532,95532,...,904373,NaN,NaN,NaN,1013280,177307,7576,27973,10565,571
mean,9.000005e+05,4.386894e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.683390e+01,-9.533849e+01,1.057711e+01,NaN,NaN,NaN,NaN,NaN,NaN
std,5.196154e+05,8.880018e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.077988e+00,1.612351e+01,4.442273e+00,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.793972e+01,-1.788026e+02,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
25%,4.500008e+05,4.842578e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.285004e+01,-1.098854e+02,8.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
50%,9.000005e+05,1.026436e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.543256e+01,-9.144000e+01,1.400000e+01,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.350000e+06,1.829796e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.083647e+01,-8.227277e+01,1.400000e+01,NaN,NaN,NaN,NaN,NaN,NaN


We see that ``fires_df`` is a combination of numeric and non-numeric data. How about the NANs?


In [11]:
# We check for % nulls

nulls = 100*fires_df.isnull().sum()/len(fires_df)

nulls

OBJECTID                       0.000000
FOD_ID                         0.000000
FPA_ID                         0.000000
SOURCE_SYSTEM_TYPE             0.000000
SOURCE_SYSTEM                  0.000000
NWCG_REPORTING_AGENCY          0.000000
NWCG_REPORTING_UNIT_ID         0.000000
NWCG_REPORTING_UNIT_NAME       0.000000
SOURCE_REPORTING_UNIT          0.000000
SOURCE_REPORTING_UNIT_NAME     0.000000
LOCAL_FIRE_REPORT_ID          77.213722
LOCAL_INCIDENT_ID             44.786500
FIRE_CODE                     82.235000
FIRE_NAME                     51.830722
ICS_209_INCIDENT_NUMBER       98.623556
ICS_209_NAME                  98.623556
MTBS_ID                       99.403000
MTBS_FIRE_NAME                99.403000
COMPLEX_NAME                  99.718111
FIRE_YEAR                      0.000000
DISCOVERY_DATE                 0.000000
DISCOVERY_DOY                  0.000000
DISCOVERY_TIME                47.666111
STAT_CAUSE_CODE                0.000000
STAT_CAUSE_DESCR               0.000000


In [12]:
# We will drop columns with more than 25% of nulls

nulls = 100*fires_df.isnull().sum()/len(fires_df)

col_to_drop = []

nulls.index

#nulls['OBJECTID']<15

for i in nulls.index:
    if nulls[i]>25:
        col_to_drop.append(i)
col_to_drop

['LOCAL_FIRE_REPORT_ID',
 'LOCAL_INCIDENT_ID',
 'FIRE_CODE',
 'FIRE_NAME',
 'ICS_209_INCIDENT_NUMBER',
 'ICS_209_NAME',
 'MTBS_ID',
 'MTBS_FIRE_NAME',
 'COMPLEX_NAME',
 'DISCOVERY_TIME',
 'CONT_DATE',
 'CONT_DOY',
 'CONT_TIME',
 'COUNTY',
 'FIPS_CODE',
 'FIPS_NAME']

In [13]:
fires_df.drop(col_to_drop, axis=1, inplace = True)

In [14]:
# Check again for nulls

nulls = 100*fires_df.isnull().sum()/len(fires_df)

nulls

OBJECTID                      0.0
FOD_ID                        0.0
FPA_ID                        0.0
SOURCE_SYSTEM_TYPE            0.0
SOURCE_SYSTEM                 0.0
NWCG_REPORTING_AGENCY         0.0
NWCG_REPORTING_UNIT_ID        0.0
NWCG_REPORTING_UNIT_NAME      0.0
SOURCE_REPORTING_UNIT         0.0
SOURCE_REPORTING_UNIT_NAME    0.0
FIRE_YEAR                     0.0
DISCOVERY_DATE                0.0
DISCOVERY_DOY                 0.0
STAT_CAUSE_CODE               0.0
STAT_CAUSE_DESCR              0.0
FIRE_SIZE                     0.0
FIRE_SIZE_CLASS               0.0
LATITUDE                      0.0
LONGITUDE                     0.0
OWNER_CODE                    0.0
OWNER_DESCR                   0.0
STATE                         0.0
Shape                         0.0
dtype: float64

Let's search for duplicates.

In [15]:
fires_df.duplicated().astype(int).sum()

0

We see that there no duplicates.


In [16]:
# THIS CELL WAS ORIGINALLY CREATED WHEN WORKING WITH 100000 OBSERVATIONS (WE ARE WORKING WITH 30% OF THE DATA NOW). WE LEAVE IT HERE IN CASE WE NEED IT WHEN WE CONSIDER THE FULL DATA SET.

# Now is time to fill in the null values.

#For  CONT_TIME and we will replace the null values with the median of the column

#fires_df["CONT_TIME"].replace(np.nan,fires_df["CONT_TIME"].median(), inplace=True)

#fires_df["CONT_DOY"].replace(np.nan,fires_df["CONT_DOY"].median(), inplace=True)


# Given that CONT_DATE seems to encode a date, we will use a ffill to replace the null vales for this column
#fires_df["CONT_DATE"].ffill(inplace=True)

#fires_df[["CONT_TIME", "CONT_DOY", "CONT_DATE"]].isnull().sum()

In [17]:
fires_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800000 entries, 0 to 1799999
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   OBJECTID                    int64  
 1   FOD_ID                      int64  
 2   FPA_ID                      object 
 3   SOURCE_SYSTEM_TYPE          object 
 4   SOURCE_SYSTEM               object 
 5   NWCG_REPORTING_AGENCY       object 
 6   NWCG_REPORTING_UNIT_ID      object 
 7   NWCG_REPORTING_UNIT_NAME    object 
 8   SOURCE_REPORTING_UNIT       object 
 9   SOURCE_REPORTING_UNIT_NAME  object 
 10  FIRE_YEAR                   int64  
 11  DISCOVERY_DATE              float64
 12  DISCOVERY_DOY               int64  
 13  STAT_CAUSE_CODE             float64
 14  STAT_CAUSE_DESCR            object 
 15  FIRE_SIZE                   float64
 16  FIRE_SIZE_CLASS             object 
 17  LATITUDE                    float64
 18  LONGITUDE                   float64
 19  OWNER_CODE           

We have no more null elements at this point. We now turn our attention to the columns that will be informative For instance, # OBJECTID can be deleted or set as index but it starts at 1!

In [18]:
fires_df.drop(['OBJECTID'], axis = 1, inplace = True)

We can store the name of the columns in our data frame

In [19]:
cols = list(fires_df.columns)
cols

['FOD_ID',
 'FPA_ID',
 'SOURCE_SYSTEM_TYPE',
 'SOURCE_SYSTEM',
 'NWCG_REPORTING_AGENCY',
 'NWCG_REPORTING_UNIT_ID',
 'NWCG_REPORTING_UNIT_NAME',
 'SOURCE_REPORTING_UNIT',
 'SOURCE_REPORTING_UNIT_NAME',
 'FIRE_YEAR',
 'DISCOVERY_DATE',
 'DISCOVERY_DOY',
 'STAT_CAUSE_CODE',
 'STAT_CAUSE_DESCR',
 'FIRE_SIZE',
 'FIRE_SIZE_CLASS',
 'LATITUDE',
 'LONGITUDE',
 'OWNER_CODE',
 'OWNER_DESCR',
 'STATE',
 'Shape']

We have to look at the glossary to see what each column contains.

In [20]:
# Let's recall what the abbreviations in our colums are  is

for names in cols:
    print(names, ':')
    print(abb.get(names))
    print('')
    print('')
    print('')

FOD_ID :
Global unique identifier



FPA_ID :
Unique identifier that contains information necessary to track back to the original record in the source dataset



SOURCE_SYSTEM_TYPE :
Type of source database or system that the record was drawn from (federal, nonfederal, or interagency)



SOURCE_SYSTEM :
Name of or other identifier for source database or system that the record was drawn from. See Table 1 in Short (2014), or \Supplements\FPAFODsourcelist.pdf, for a list of sources and their identifier



NWCG_REPORTING_AGENCY :
Active National Wildlife Coordinating Group (NWCG) Unit Identifier for the agency preparing the fire report (BIA = Bureau of Indian Affairs, BLM = Bureau of Land Management, BOR = Bureau of Reclamation, DOD = Department of Defense, DOE = Department of Energy, FS = Forest Service, FWS = Fish and Wildlife Service, IA = Interagency Organization, NPS = National Park Service, ST/C&L = State, County, or Local Organization, and TRIBE = Tribal Organization)



NWCG_REPORT

We will drop ``FPA_ID`` and ``Shape``, as we think they contain no relevant information to answer the questions we are interested in

In [21]:
# We will drop FPA_ID and Shape, as we think they contain no relevant information to answer the questions we are interested in

fires_df.drop(['FPA_ID'], axis = 1, inplace = True)
fires_df.drop(['Shape'], axis = 1, inplace = True)

Below we have a list of the remaining columns that I think should not have a predictive power on the cause of a fire. 

In [22]:
non_relevant_col = ['FOD_ID', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_NAME','NWCG_REPORTING_UNIT_ID', 'SOURCE_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT']

In [23]:
for col in non_relevant_col:
    fires_df.drop([col], axis =1, inplace = True)

Let us see what our dataframe looks like so far

In [24]:
fires_df.head().T

,0,1,2,3,4
SOURCE_SYSTEM_TYPE,FED,FED,FED,FED,FED
SOURCE_SYSTEM,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT
FIRE_YEAR,2005,2004,2004,2004,2004
DISCOVERY_DATE,2.4534e+06,2.45314e+06,2.45316e+06,2.45318e+06,2.45318e+06
DISCOVERY_DOY,33,133,152,180,180
STAT_CAUSE_CODE,9,1,5,1,1
STAT_CAUSE_DESCR,Miscellaneous,Lightning,Debris Burning,Lightning,Lightning
FIRE_SIZE,0.1,0.25,0.1,0.1,0.1
FIRE_SIZE_CLASS,A,A,A,A,A
LATITUDE,40.0369,38.9331,38.9842,38.5592,38.5592


The next column we have to take care of is ``DISCOVERY_DATE``. Note that the format is not the ususal one.

In [25]:
# We can see the column of interest with 
fires_df.loc[:10, 'DISCOVERY_DATE']

0     2453403.5
1     2453137.5
2     2453156.5
3     2453184.5
4     2453184.5
5     2453186.5
6     2453187.5
7     2453437.5
8     2453444.5
9     2453187.5
10    2453188.5
Name: DISCOVERY_DATE, dtype: float64

After some search in the web, we found out that this notation is called Julian Date and it is commonly used in software. See for instance: https://en.wikipedia.org/wiki/Julian_day.

To make the conversion to Gregorian calendar date, we follow the directions given in: https://oneau.wordpress.com/2011/08/30/jdcal/

We start from getting the column with the dates

In [26]:
from jdcal import jd2gcal

jd_col = fires_df['DISCOVERY_DATE']

Let's see an example:

In [27]:
#Let's see an example:
z = fires_df.loc[1,'DISCOVERY_DATE']

z

2453137.5

In [28]:
# To obtain the integer and fraction part of the Julian dates we will use modf

import math

x = math.modf(z)

x

(0.5, 2453137.0)

Below we explain the procedure to convert Julian dates to Gregorian dates. We do it with only one example and then we will generalize it to all the column

In [29]:
# Let's look at an example to explain what we will do in the next line of code

#This turns the Julian date into Gregorian date
print('Gregorian date:', jd2gcal(x[0], x[1]))

# We can obtain the year with
print('Year:', jd2gcal(x[0], x[1])[0])

# We can obtain the month with
print('Month:',jd2gcal(x[0], x[1])[1])

# We can obtain the day with
print('Day:', jd2gcal(x[0], x[1])[2])

#  We are not interested in the fraction of the day the fire was discovered, so we will not save this piece of information.

Gregorian date: (2004, 5, 12, 0.0)
Year: 2004
Month: 5
Day: 12


Note that May $12^{th}$, $2004$ is the $133^{rd}$ day of that year. This coincides with the information given in the dataset. We are ready to convert all the Julian dates to Gregorian.

In [30]:
%%time
# Now we repeat the process described above with every element of the Julian date column
gcal_year_col = []
gcal_month_col = []
gcal_day_col = []
for jdate in jd_col:
    #We first convert jdata into a float
    jdate = float(jdate)
    #Now we separate the integer part from the fractional part
    jdate = math.modf(jdate)
    #Finally, we convert from Julian to Gregorian date and we store Year, Month and Date in different columns
    gcal_year_col.append(jd2gcal(jdate[0],jdate[1])[0])
    gcal_month_col.append(jd2gcal(jdate[0],jdate[1])[1])
    gcal_day_col.append(jd2gcal(jdate[0],jdate[1])[2])

Wall time: 44.2 s


The next step is to add the newly created columns to our date set. After that,  I will drop ``DISCOVERY_DATE``

In [31]:
# The next step is to add the newly created columns to our date set. 

fires_df['Discovery_Year'] = gcal_year_col
fires_df['Discovery_Month'] = gcal_month_col
fires_df['Discovery_Day'] = gcal_day_col

In [32]:
# Now we can drop DISCOVERY_DATE column

fires_df.drop(['DISCOVERY_DATE'], axis = 1, inplace = True)

In [33]:
# It's time to take a look at our dataset again

fires_df.head().T

,0,1,2,3,4
SOURCE_SYSTEM_TYPE,FED,FED,FED,FED,FED
SOURCE_SYSTEM,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT
FIRE_YEAR,2005,2004,2004,2004,2004
DISCOVERY_DOY,33,133,152,180,180
STAT_CAUSE_CODE,9,1,5,1,1
STAT_CAUSE_DESCR,Miscellaneous,Lightning,Debris Burning,Lightning,Lightning
FIRE_SIZE,0.1,0.25,0.1,0.1,0.1
FIRE_SIZE_CLASS,A,A,A,A,A
LATITUDE,40.0369,38.9331,38.9842,38.5592,38.5592
LONGITUDE,-121.006,-120.404,-120.736,-119.913,-119.933


It would be nice to see how different ``Discovery_Year`` and ``FIRE_YEAR `` are.

In [34]:
diff = fires_df['Discovery_Year']- fires_df['FIRE_YEAR']

diff.sum()

2

They almost the same information. This is just telling us that the year a particlar fire started was the same year it was discovered. This makes sense, though it may not always be the case as some fires could be produce at the end of December the $31^{ft}$ of a particular year and it can be dicover the next year. We will eliminate ``FIRE_YEAR``

In [35]:
fires_df.drop(['FIRE_YEAR'], axis = 1, inplace = True)

Notice that ``DISCOVERY_DOY`` is a column colpletely detemined by the newly added columns: ``Discovery_Year``, ``Discovery_Month``, and ``Discovery_Day``. Therefore, we will also eliminate it.

In [36]:
fires_df.drop(['DISCOVERY_DOY'], axis = 1, inplace = True)

Again, we can see what our dataframe looks like so far

In [37]:
fires_df.head().T

,0,1,2,3,4
SOURCE_SYSTEM_TYPE,FED,FED,FED,FED,FED
SOURCE_SYSTEM,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT,FS-FIRESTAT
STAT_CAUSE_CODE,9,1,5,1,1
STAT_CAUSE_DESCR,Miscellaneous,Lightning,Debris Burning,Lightning,Lightning
FIRE_SIZE,0.1,0.25,0.1,0.1,0.1
FIRE_SIZE_CLASS,A,A,A,A,A
LATITUDE,40.0369,38.9331,38.9842,38.5592,38.5592
LONGITUDE,-121.006,-120.404,-120.736,-119.913,-119.933
OWNER_CODE,5,5,13,5,5
OWNER_DESCR,USFS,USFS,STATE OR PRIVATE,USFS,USFS


We will now download the dataframe as it is so that we can work with it later without having to run all the cell above again. That will save us time in the future.

In [38]:
fires_df.count()

SOURCE_SYSTEM_TYPE    1800000
SOURCE_SYSTEM         1800000
STAT_CAUSE_CODE       1800000
STAT_CAUSE_DESCR      1800000
FIRE_SIZE             1800000
FIRE_SIZE_CLASS       1800000
LATITUDE              1800000
LONGITUDE             1800000
OWNER_CODE            1800000
OWNER_DESCR           1800000
STATE                 1800000
Discovery_Year        1800000
Discovery_Month       1800000
Discovery_Day         1800000
dtype: int64

In [39]:
%%time
fires_df.to_csv(r'C:\Users\francisco\Desktop\BrainStation\Capstoneproject_BS\FiresUSA\Fires_pandas_df.csv', index = False)

Wall time: 1min 41s


We can read in the file again.

In [40]:
%%time
fires_df_ = pd.read_csv('C:\\Users\\francisco\\Desktop\\BrainStation\\Capstoneproject_BS\\FiresUSA\\Fires_pandas_df.csv')

Wall time: 5.16 s


This is certancly much faster than reading in the data as before. Let's see if we have the same number of rows as before.

In [41]:
fires_df_.count()

SOURCE_SYSTEM_TYPE    1800000
SOURCE_SYSTEM         1800000
STAT_CAUSE_CODE       1800000
STAT_CAUSE_DESCR      1800000
FIRE_SIZE             1800000
FIRE_SIZE_CLASS       1800000
LATITUDE              1800000
LONGITUDE             1800000
OWNER_CODE            1800000
OWNER_DESCR           1800000
STATE                 1800000
Discovery_Year        1800000
Discovery_Month       1800000
Discovery_Day         1800000
dtype: int64

 We will finish this notebook here and we will proceed with another note book to make the modelling.